In [47]:
import env
import pandas as pd
import pydataset as data

In [11]:
import env

def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

df = pd.read_sql('SELECT * FROM attendance', get_connection('tidy_data'))

df.head()


,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [14]:
df = df.rename(columns={'Unnamed: 0': 'student'})

In [23]:
df = df.melt(id_vars=['student'], var_name='date')
df

/var/folders/2j/32g17nrx0rj4vvx130_l42kw0000gn/T/ipykernel_12657/1476348290.py:1: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  df = df.melt(id_vars=['student'], var_name='date')


,student,date,value
0,Sally,date,date
1,Jane,date,date
2,Billy,date,date
3,John,date,date
4,Sally,date,date
...,...,...,...
763,John,value,1.0
764,Sally,value,0.9
765,Jane,value,0.9
766,Billy,value,0.9


In [24]:
def attendance_grade(letter):
    if letter == 'P':
        return 1
    elif letter == 'A':
        return 0
    elif letter == 'H':
        return 0.5
    elif letter == 'T':
        return 0.9
attendance_grade('H')

0.5

In [25]:
df['grade'] = df.value.apply(attendance_grade)
df.groupby('student').grade.mean()

student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64

In [28]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

df_coffee = pd.read_sql('SELECT * FROM coffee_levels', get_connection('tidy_data'))

In [29]:
df_coffee

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [30]:
df_coffee = df_coffee.pivot_table(values='coffee_amount', index='hour', columns='coffee_carafe')

In [31]:
df_coffee

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


In [36]:
query = '''
        select * from coffee_levels'''
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
df_cake = pd.read_sql('SELECT * FROM cake_recipes', get_connection('tidy_data'))
df_cake

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [39]:
recipes_and_positions = df_cake['recipe:position'].str.split(':', expand=True)
recipes_and_positions.columns = ['recipe', 'position']
df_cake = pd.concat([df_cake, recipes_and_positions], axis=1).drop(columns='recipe:position')
df_cake = df_cake.melt(id_vars=['recipe', 'position'], var_name='temp', value_name='tastiness')
df_cake.head()

KeyError: 'recipe:position'

In [40]:
df_cake

,recipe,position,temp,tastiness
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207
5,c,top,225,71.306308
6,d,bottom,225,52.799753
7,d,top,225,96.873178
8,a,bottom,250,53.912627
9,a,top,250,52.009735


In [41]:
df_cake.groupby('recipe').tastiness.mean()

recipe
a    63.922201
b    76.736074
c    75.874748
d    62.864844
Name: tastiness, dtype: float64

In [42]:
df_cake.groupby('temp').tastiness.mean()

temp
225    71.306022
250    66.577437
275    74.886754
300    66.627655
Name: tastiness, dtype: float64

In [43]:
df_cake.sort_values(by='tastiness').tail()

,recipe,position,temp,tastiness
4,c,bottom,225,96.470207
7,d,top,225,96.873178
19,b,top,275,98.594881
24,a,bottom,300,98.786784
26,b,bottom,300,99.248541


FileNotFoundError: [Errno 2] No such file or directory: 'attendance.csv'